# Wokflow  **TEST** con Full Bayesiana 

## Inicializacion

In [1]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-11-15 16:11:42 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,658633,35.2,1454648,77.7,1227762,65.6
Vcells,1222293,9.4,8388608,64.0,1975146,15.1


In [2]:
plocal <- list()

# 501
plocal$qcanaritos <- 5L
plocal$min_data_in_leaf <- 20L
plocal$learning_rate <- 1.0
plocal$gradient_bound <- 0.1

plocal$APO <- 5
plocal$ksemillerio <- 1


In [17]:
source("/home/vickydiliscia/dmeyf2025/src/churn/C2020/config/_index.R")

In [18]:
source(file.path(PARAM$paths$functions, "index.R"))
source_functions(PARAM$paths$functions)

Sourcing: add_divisiones.R

Sourcing: add_ratio.R

Sourcing: agregar_clase_ternaria.R

Sourcing: agregar_var_rf.R

Sourcing: aplicar_dataquality_zeros.R

Sourcing: apply_sum_specs_multi.R

Sourcing: audit_nullear.R

Sourcing: build_variables_rotas.R

Sourcing: crear_proporcion.R

Sourcing: features_historicos_optimizados.R

Sourcing: features_historicos.R

Sourcing: flip.R

Sourcing: interpolar_faltantes.R

Sourcing: lags.R

Sourcing: normalizar_ctrx_quarter.R

Sourcing: print_utils.R

Sourcing: run_features_historicas.R

Sourcing: tendencias_y_mas.R

Sourcing: ver_meses_rotos.R

✅ Todas las funciones fueron cargadas correctamente.



In [8]:
setwd("/content/buckets/b1/exp")
experimento_folder <- PARAM$experimento
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

## Preprocesamiento

### Generacion de la clase_ternaria

In [ ]:
Sys.time()
require( "data.table" )
dataset <- fread(PARAM$paths$datasetcrudo)

dataset <- agregar_clase_ternaria(dataset)

setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

### Eliminacion de Features

In [12]:
# Salsa Magica para 202106
dataset[, mprestamos_personales := NULL ]
dataset[, cprestamos_personales := NULL ]

### Data Quality

#### Flip Internet

In [13]:
flip(dataset, internet, month_to = 202009L)
Sys.time()

[1] "2025-11-15 16:14:52 UTC"

#### Tratamiento variables con meses rotos

In [15]:
variables_rotas <- build_variables_rotas(dataset)

if (PARAM$dataquality$zeros == "nullear") {
  dataset <- aplicar_dataquality_zeros(
    dataset,
    variables_rotas,
    modo = "nullear"
  )

} else if (PARAM$dataquality$zeros == "interpolar") {
  dataset <- aplicar_dataquality_zeros(
    dataset,
    variables_rotas,
    modo = "interpolar",
    treat_zero_as_na = TRUE,
    require_both     = TRUE,
    only_if_na       = TRUE,
    integer_prefix   = "^c",
    round_integers   = TRUE
  )

} else {
  message("No se aplicó tratamiento de dataquality.")
}

Sys.time()

[1] "2025-11-15 16:16:02 UTC"

In [ ]:
if (PARAM$dataquality$zeros == "nullear") {
# Si usaste replace_all_in_month = FALSE (nullear solo ceros):
res <- audit_nullear(dataset, variables_rotas, month_col = "foto_mes",
                     replace_all_in_month = FALSE)

# Ver primeras filas y variables con problemas (si las hubiera)
res[ok_sin_ceros_marc | status == "ok"][]
}else{
    message("No se audita")
}

### Feature Engineering Intra-Mes

#### Variables Sugeridas

In [19]:
if (isTRUE(PARAM$FE_intrames$kmes)) {
  dataset[, kmes := foto_mes %% 100]
  message("Columna kmes creada")
} else {
  message("Columna kmes no creada")
}

if (isTRUE(PARAM$FE_intrames$normquarter)) {
  dataset <- normalizar_ctrx_quarter(dataset)
  message("Normalizacion ctrx_quarter aplicada")
} else {
  message("Normalizacion ctrx_quarter no aplicada")
}

if (isTRUE(PARAM$FE_intrames$ratio_payroll_edad)) {
    dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]  
    message("Columna mpayroll_sobre_edad creada")
} else {
    message("Columna mpayroll_sobre_edad creada")
}

Sys.time()

Columna kmes creada

Normalizacion ctrx_quarter aplicada

Columna mpayroll_sobre_edad creada



[1] "2025-11-15 16:18:06 UTC"

#### Sumas de Variables

In [20]:
spec_sumas <- fread(PARAM$FE_intrames$specsumas)

# Normalizar nombres de columnas
setnames(spec_sumas, tolower(names(spec_sumas)))
req <- c("new_var","inputs","active")
faltan <- setdiff(req, names(spec_sumas))
if (length(faltan)) stop("Faltan columnas en spec: ", paste(faltan, collapse=", "))

# Limpiar espacios en inputs y new_var
spec_sumas[, new_var := trimws(new_var)]
spec_sumas[, inputs  := trimws(inputs)]

# 2) Convertir 'active' a lógico robusto
to_bool <- function(x){
  if (is.logical(x)) return(x)
  x2 <- tolower(trimws(as.character(x)))
  x2 %in% c("true","t","1","yes","y","si","sí")
}
spec_sumas[, active := to_bool(active)]

dataset <- apply_sum_specs_multi(
  dataset,
  spec = spec_sumas,
  from_file = FALSE,
  sep_inputs = ","
)
Sys.time()

ERROR: Error in fread(PARAM$FE_intrames$specsumas): File '/home/vickydiliscia/dmeyf2025/src/churn/C2020/featurespecs/spec_sumas.csv' does not exist or is non-readable. getwd()=='/home/vickydiliscia/buckets/b1/exp/APO2020-202106'


### Feature Engineering Historico

#### Lags y Delta Lags

In [ ]:
setorder(dataset, numero_de_cliente, foto_mes)

if (isTRUE(PARAM$FE_hist$Lags$Run)) {
  dataset <- crear_lags(dataset, lags = PARAM$FE_hist$Lags$Order)
} else {
  message("Lags no aplicados")
}

if (isTRUE(PARAM$FE_hist$DeltaLags$Run)) {
  dataset <- crear_delta_lags(dataset, lags = PARAM$FE_hist$DeltaLags$Order)
} else {
  message("Delta Lags no aplicados")
}

Sys.time()

[1] "2025-11-13 05:21:56 UTC"

#### Tendencias 6 meses

In [ ]:

cols_lagueables <- intersect(cols_lagueables, colnames(dataset))
setorder(dataset, numero_de_cliente, foto_mes)

if( PARAM$FE_hist$Tendencias$run) {
    tendencias_y_mas(dataset,
    cols = cols_lagueables,
    ventana = PARAM$FE_hist$Tendencias$ventana, 
    tendencia = PARAM$FE_hist$Tendencias$tendencia,
    minimo = PARAM$FE_hist$Tendencias$minimo,
    maximo = PARAM$FE_hist$Tendencias$maximo,
    promedio = PARAM$FE_hist$Tendencias$promedio,
    ratioavg = PARAM$FE_hist$Tendencias$ratioavg,
    ratiomax = PARAM$FE_hist$Tendencias$ratiomax
  )
}

ncol(dataset)
Sys.time()

#### Ratios, Medias moviles, Desvios y Streaks historicos

In [ ]:
# Tomo specs de que variables crear
spec_hist <- fread(PARAM$FE_hist$spechistoricas) 

# Aplico variables
apply_rolling_with_kernel(dataset, spec_hist)

Sys.time()

### Feature Engineering a partir de hojas de Random Forest

In [ ]:
if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

In [ ]:
# Agregar Variables de Random Forest

agregar_var_rf(PARAM)

Sys.time()

In [ ]:
# guardar_db(dataset, "/home/vickydiliscia/datasets/dataset_APO_1")

## Modelado

No hay modelado, no se hace optimizacion de hiperparametros.

## Produccion

Las decisiones que se toman para la construccion del modelo final son:
* Los positvos son  POS={"BAJA+1", "BAJA+2"}, esta es una meticulosa decisión.
* Se entrena en los treinta meses del intervalo [201901, 202104]
* Se realiza undersampling al 5%
* Se utilizan los hiperparámetros optimos encontrados en la Bayesian Optimization
   * Se escala min_data_in_leaf

### Final Training Strategy

In [ ]:
# se filtran los meses donde se entrena el modelo final
dataset_train_final <- dataset[foto_mes %in% PARAM$train_final$training]

In [ ]:
# canaritos
PARAM$train_final$lgbm$qcanaritos <- plocal$qcanaritos

cols0 <- copy(colnames(dataset_train_final))
filas <- nrow(dataset_train_final)

if( PARAM$train_final$lgbm$qcanaritos > 0 ) {
  for( i in seq(PARAM$train_final$lgbm$qcanaritos) ){
    dataset_train_final[, paste0("canarito_",i) := runif( filas) ]
  }

  # las columnas canaritos mandatoriamente van al comienzo del dataset
  cols_canaritos <- copy( setdiff( colnames(dataset_train_final), cols0 ) )
  setcolorder( dataset_train_final, c( cols_canaritos, cols0 ) )
}

Sys.time()

[1] "2025-11-13 05:22:54 UTC"

#### Registros cambio las proporciones de POS/NEG

In [ ]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train_final[, azar := runif(nrow(dataset_train_final))]
dataset_train_final[, training := 0L]

dataset_train_final[
  (azar <= PARAM$train_final$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

dataset_train_final[, azar:= NULL] # elimino la columna azar

### Target Engineering

In [ ]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train_final[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

### Final Models

Aqui SIEMPRE voy a hacer un semillerio, independientemente de si en la Bayesian Optimization calculé un semillerio en cada iteración.
<br> Entreno un LightGBM para cada semilla,  y guardo el modelo dentro de la carpeta  **modelitos**
<br> Intencionalmente en una primera etapá se generan los modelos y graban, y en una segunda etapa se leen eso modelos y se aplican a los datos del futuro

APO controla cuantas veces se repite el modelo, que se usa para promediar ganancias y reportar en la Pseudo Competencia algo razonable
<br> El modelo puede ser un LightGBM simple (ksemillerio==1)  o un Ensemble Semillerio( ksemillerio > 1 )
<br> Lamentablmente APO necesita utilizar muchas semillas, y eso demanda TIEMPO de corrida

In [ ]:

PARAM$train_final$lgbm$param_completo <-  list(
  boosting= "gbdt",
  objective= "binary",
  metric= "custom",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE,
  verbosity= -100,

  seed= PARAM$semilla_primigenia,

  max_bin= 31L,
  min_data_in_leaf= plocal$min_data_in_leaf,  #este ya es el valor default de LightGBM

  num_iterations= 9999L, # dejo libre la cantidad de arboles, zLightGBM se detiene solo
  num_leaves= 9999L, # dejo libre la cantidad de hojas, zLightGBM sabe cuando no hacer un split
  learning_rate= plocal$learning_rate,  # se lo deja en 1.0 para que si el score esta por debajo de gradient_bound no se lo escale
    
  feature_fraction= 0.50, # un valor equilibrado, habra que probar alternativas ...
    
  canaritos= PARAM$train_final$lgbm$qcanaritos, # fundamental en zLightGBM, aqui esta el control del overfitting
  gradient_bound= plocal$gradient_bound   # default de zLightGBM
)

Sys.time()

[1] "2025-11-13 05:22:54 UTC"

In [ ]:
# Semillerio Final
PARAM$train_final$APO <- plocal$APO
PARAM$train_final$ksemillerio  <- plocal$ksemillerio

PARAM$train_final$cortes <- c(8000, 8500, 9000, 9500, 10000, 10500, 11000, 11500, 12000)

In [ ]:
# detach("package:lightgbm", unload= TRUE)

In [ ]:
if( !require("zlightgbm") ) install.packages("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/zlightgbm_4.6.0.99.tar.gz", repos= NULL, type= "source")
require("zlightgbm")

Loading required package: zlightgbm



In [ ]:
if(!require("primes")) install.packages("primes")
require("primes")

Loading required package: primes



In [ ]:
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
PARAM$train_final$semillas <- sample(primos)[seq( PARAM$train_final$APO*PARAM$train_final$ksemillerio )]
PARAM$train_final$semillas

[1] 974411

In [ ]:
campos_buenos <- setdiff(
  colnames(dataset_train_final),
  c( "clase_ternaria", "clase01", "training", "azar")
)

In [ ]:
# dejo los datos en formato LightGBM
dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

filas 235841 columnas 925 


[1] "2025-11-13 05:22:57 UTC"

In [ ]:
# genero los modelitos
dir.create( "modelitos", showWarnings= FALSE)

param_completo <- copy( PARAM$train_final$lgbm$param_completo)

for( sem in PARAM$train_final$semillas ) {

  arch_modelo <- paste0("./modelitos/mod_", sem, ".txt")
  if( !file.exists( arch_modelo ) )
  {
    param_completo$seed <- sem

    modelito <- lgb.train(
      data= dtrain_final,
      param= param_completo
    )

    lgb.save( modelito, filename= arch_modelo)
    rm(modelito)
    gc()
  }
}

Sys.time()

[1] "2025-11-13 05:34:20 UTC"

### Scoring

Se hace el predict() del modelo en los datos del futuro

In [ ]:
dfuture <- dataset[foto_mes %in% PARAM$train_final$future ]

cols0 <- copy(colnames(dfuture))
filas <- nrow(dfuture)

if( PARAM$train_final$lgbm$qcanaritos > 0 ) {
  for( i in seq(PARAM$train_final$lgbm$qcanaritos) ){
    dfuture[, paste0("canarito_",i) := runif( filas) ]
  }

  # las columnas canaritos mandatoriamente van al comienzo del dataset
  cols_canaritos <- copy( setdiff( colnames(dfuture), cols0 ) )
  setcolorder( dfuture, c( cols_canaritos, cols0 ) )
}

In [ ]:
# dataset de future, donde en este caso estoy haciendo testing

mfuture <- data.matrix(dfuture[, campos_buenos, with= FALSE])

dfuture[, ganancia := ifelse(clase_ternaria=="BAJA+2", 780000, -20000)]

In [ ]:
mganancias <- matrix( nrow=PARAM$train_final$APO, ncol= length(PARAM$train_final$cortes) )

if( file.exists("prediccion.txt") )
  file.remove("prediccion.txt")

[1] TRUE

In [ ]:
# aplico el modelo a los datos del future

for( vapo in seq(PARAM$train_final$APO) ) {
  # inicializacion en CERO
  vpred_acum <- rep(0.0, nrow(dfuture))
  qacumulados <- 0

  desde <- 1 + (vapo-1)*PARAM$train_final$ksemillerio
  hasta <- desde + PARAM$train_final$ksemillerio - 1
  semillas <- PARAM$train_final$semillas[desde:hasta]

  for( sem in semillas ) {

    arch_modelo <- paste0("./modelitos/mod_", sem, ".txt")
    if( file.exists( arch_modelo ) )
    {
      modelo_final <- lgb.load(arch_modelo) # leo del disco
      # hago el predict() y acumulo
      vpred_acum <- vpred_acum + predict(modelo_final, mfuture)
      qacumulados <- qacumulados + 1
      rm(modelo_final)
      gc()
    }
  }

  if( qacumulados > 0 ) {
    vpred_acum <- vpred_acum / qacumulados  # paso a probabildiad
    # tabla de prediccion, puede ser util para futuros ensembles
    #  ya que le modelo ganador va a ser un ensemble de LightGBMs

    tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes, ganancia)]
    tb_prediccion[, meta_modelo := vapo]
    tb_prediccion[, prob := vpred_acum ]
    setorder( tb_prediccion, -prob )
    tb_prediccion[, gan_acum := cumsum(ganancia)]
    tb_prediccion[, ganancia := NULL ]

    # acumulo las ganancias
    for( icor in seq(length(PARAM$train_final$cortes)) ){
      mganancias[ vapo, icor ] <- tb_prediccion[ PARAM$train_final$cortes[icor], gan_acum ]
    }

    # grabo las probabilidades del modelo
    fwrite(tb_prediccion,
      file= "prediccion.txt",
      sep= "\t",
      append= TRUE
    )

    rm(tb_prediccion)
    gc()
  }
}

Sys.time()

[1] "2025-11-13 05:34:30 UTC"

In [ ]:
mganancias

402400000,403600000,4.04e+08,403600000,406400000,405200000,4.04e+08,398800000,4.04e+08


### Clasificacion

Se tomó la decisión de enviar a los 11000 registros con mayor probabilidad de POS={"BAJA+1","BAJA+"}
<br> esto se determinó en forma artesanal analizando meses anterior
<br> esta es una muy importante decisión 

In [ ]:
# genero archivos con los  "envios" mejores
dir.create("kaggle", showWarnings=FALSE)

tb_prediccion <- fread("prediccion.txt")

In [ ]:
# genero archivos de fantasia, que NO son el que voy a subir a la Pseudo Competencia Kaggle
envios <- 11000

for( vapo in seq(PARAM$train_final$APO) ) {
  if( tb_prediccion[meta_modelo==vapo, .N] > 0 ) {
    tb_pred <- tb_prediccion[meta_modelo==vapo]
    setorder( tb_pred, -prob )
    tb_pred[, Predicted := 0L] # seteo inicial a 0
    tb_pred[1:envios, Predicted := 1L] # marco los primeros

    archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", vapo, "_", envios, ".csv")

    # grabo el archivo
    fwrite(tb_pred[, list(numero_de_cliente, Predicted)],
      file= archivo_kaggle,
      sep= ","
    )

    rm( tb_pred )
    gc()
  }
}

Sys.time()

[1] "2025-11-13 05:34:32 UTC"

### Subida a Pseudo Kaggle

Aqui viene la verdadera magia de  APO = A Prueba Overfiteres
<br>Se sube un submit a la Pseudo Competencia Kaggle cuya ganancia coincide casi exactamente con la ganancia MEDIA , promediada  APO veces

In [ ]:
colmedias <- colMeans( mganancias, na.rm=TRUE )
mcorte_mejor <- max(colmedias, na.rm=TRUE)
icorte_mejor <- which.max( colmedias )
corte_mejor <- PARAM$train_final$cortes[icorte_mejor]

In [ ]:
tbl <- as.data.table( as.list( colmedias ) )
colnames(tbl) <- paste0( "e", PARAM$train_final$cortes )
tbl[, experimento := PARAM$experimento ]

exp_gral <- "/content/buckets/b1/exp/apo-gral"
dir.create(exp_gral, showWarnings=FALSE)
fwrite( tbl,
  file= paste0( exp_gral, "/tb_experimentos.txt"),
  sep= "\t",
  append= TRUE
)

In [ ]:
colnames( mganancias ) <- paste0( "e", PARAM$train_final$cortes )
tbl_local <- as.data.table( mganancias )

fwrite( tbl_local,
  file= "tb_apo.txt",
  sep= "\t"
)

In [ ]:
icerca <- which.min(  abs( tb_prediccion$gan_acum - mcorte_mejor ) )
vmodelo <- tb_prediccion[ icerca, meta_modelo ]
tb_pred <- tb_prediccion[meta_modelo==vmodelo]

mcorte_mejor
icerca
tb_prediccion[ icerca]

[1] 406400000

[1] 9720

numero_de_cliente,foto_mes,meta_modelo,prob,gan_acum
<int>,<int>,<int>,<dbl>,<dbl>
844348099,202106,1,0.02423331,406400000


In [ ]:
icerca <- which.min(  abs( tb_pred$gan_acum - mcorte_mejor ) )
icerca

[1] 9720

In [ ]:
icerca <- which.min(  abs( tb_prediccion$gan_acum - mcorte_mejor ) )
vmodelo <- tb_prediccion[ icerca, meta_modelo ]
tb_pred <- tb_prediccion[meta_modelo==vmodelo]

icerca <- which.min(  abs( tb_pred$gan_acum - mcorte_mejor ) )
tb_pred[, Predicted := 0L] # seteo inicial a 0
tb_pred[1:icerca, Predicted := 1L] # marco los primeros

archivo_pseudo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_",  icerca, ".csv")

# grabo el archivo
fwrite(tb_pred[, list(numero_de_cliente, Predicted)],
  file= archivo_pseudo_kaggle,
  sep= ","
)


In [ ]:
# la subida a Kaggle
comando <- "kaggle competitions submit"
competencia <- "-c  test-202106"
arch <- paste( "-f", archivo_pseudo_kaggle)
mensaje <-  paste0( "-m 'exp=", PARAM$experimento,
  "  ", paste(names(plocal), plocal, sep= "=", collapse= ";" ),
  " envios=", icorte_mejor,"'")

                    
linea <- paste( comando, competencia, arch, mensaje)
salida <- system(linea, intern=TRUE)
cat(salida)

Successfully submitted to Test202106

In [ ]:
Sys.time()

[1] "2025-11-13 05:34:35 UTC"